In [1]:
%pylab
import os, re, time, sys
#import cPickle as pickle
from PIL import Image

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# This part deals with setting the paths for each computer / os
# Linux and Windows have different syntax for file paths and we want compatibility with both
# So, file paths are stored in a file called :
# declic_settings.txt
# located in the SAME directory/folder as this .ipynb file
# There should be 2 paths set up in the following way:
# save_path = 'insert directory you want to save to here'
# im_path = 'insert directory your images are coming from here'
# On Windows, you need to represent \ in a file path as \\
# Saves correct path strings in variables called:
# save_path, im_path

from sys import platform
if platform == "linux" or platform == "linux2":
    settings_path = "~/declic_settings"
elif platform == "darwin":
    settings_path = "~/declic_settings.txt"
elif platform == "win32":
    path = os.path.abspath('.')
    settings_path = os.path.expanduser(path)+u'\\declic_settings.txt'
    save_path, im_path = False, False
    f = open(settings_path,'r')
    f = list(f)
    for line in f:
        try:
            line = line.replace(' ','').replace('\n','').replace("'","").replace('"','')
            line = line.replace('\\\\','\\').split('=')
            varname, varval = line
            varname = varname.replace('\n','')
            varval = varval.replace('\n','').replace("'","")
            if varname == "save_path": save_path = varval
            elif varname == "im_path": im_path = varval
        except:
            print sys.exc_info()
            print "bad path definition file or not found!"

    if not (save_path and im_path): print "error reading path variables"

In [ ]:
def gen_tnsp(all_names): #what is tnsp short for? recently added process
    #just extract and save cell centers sequentially 
    
    for n in range(len(all_names)):
        thisname = all_names[n]
        chrono = time.time()
        I = np.array(Image.open(thisname))/255.0
        plt.figure()
        plt.imshow(I)
        plt.savefig('{:s}image{:03d}_.png'.format(save_path,n), dpi=300)
        plt.close()
        chrono = time.time()-chrono
        if n%10==0: print 'treating {:d} of {:d} images, at {:0.1f} seconds per image. Approx {:0.1f} minutes remaining'.format(n, len(all_names), chrono, (len(all_names)-n)*chrono/60)
        #tick = re.findall('[0-9]{9,10}', thisname)[0]
        #resname = 'file_{:04d}_centres_from_im_{:s}_.pkl'.format(n, tick)
        #pickle.dump((centers, thisname), open('{:s}{:s}'.format(res_path,resname), 'wb'))
        


start_tick, end_tick = 1476315641, 1477017240 #start/end of img seq: manual add

#scientific parameters
mouteur = 1476095627 #from manips file!        
        
names = os.listdir(im_path)#make a list of files, modified for selection
names = [i for i in names if '1.bmp' in i]
names.sort() #need to put in order
temp_names = []
times = []
    
for n in names: #step through list and save tick 
    try:
        tick = int(re.findall('[0-9]{9,10}',n)[0]) #9-10 numeric characters
        if tick >= start_tick:
            temp_names.append(n)
            times.append((tick - mouteur)/23.0) #sec diff, incl prime number in system
        if tick > end_tick: break
    except: continue
names = ['{}{}'.format(im_path,n,) for n in temp_names] #modified for selection

times = np.array(times)

gen_tnsp(names)


treating 0 of 771 images, at 0.8 seconds per image. Approx 10.7 minutes remaining
treating 10 of 771 images, at 0.8 seconds per image. Approx 10.3 minutes remaining
treating 20 of 771 images, at 0.8 seconds per image. Approx 10.0 minutes remaining
